In [1]:
import pandas as pd
import numpy as np
from DatabaseMergingFunctions import targetLibToDF as tl
from logFunctions import write_log
from datetime import datetime

In [2]:
'''
This program is used to make the decoy for the spectral library. We plan to use SpectraST precursor swap technique for this purpose
Let M be the precursor mass of the target spectrum, search the spectra of range M +/- 8Da
Only consider spectra with the same charge state of the target spectrum
To define the decoy spectrum: select the spectrum with highest precursor mass, and assign M as its precursor
Rational: in this way, each target spectrum will have a corresponding decoy spectrum but with the exact same precursor mass (but a different MS2 pattern)

'''


'\nThis program is used to make the decoy for the spectral library. We plan to use SpectraST precursor swap technique for this purpose\nLet M be the precursor mass of the target spectrum, search the spectra of range M +/- 8Da\nOnly consider spectra with the same charge state of the target spectrum\nTo define the decoy spectrum: select the spectrum with highest precursor mass, and assign M as its precursor\nRational: in this way, each target spectrum will have a corresponding decoy spectrum but with the exact same precursor mass (but a different MS2 pattern)\n\n'

In [3]:
target_lib = "../../Decoy_Precursor_Swap/intermediate/jumplib_human_tmt18_default_target.splib"

In [4]:
targetDF = tl(target_lib)

In [14]:
test_increasing = targetDF.sort_values(by=["precursorMZ"], ascending=True)
test_decreasing = targetDF.sort_values(by=["precursorMZ"], ascending=True)

In [16]:
check = 1103.1334053

In [19]:
check_shift = 1103.1334053+8

In [27]:
z = str(2)

In [40]:
test.query('charge == {} & precursorMZ > {}'.format(z, check_shift)).head(1)

,scan,charge,[M+H]+,precursorMZ,L_ID,L_peptide,L_protein,mod_pos_line,spectrum_line,jscore_line,max_JScore,psm_no_line,RT,Lib_Description,Total_Batches,Peptide_observed,m/z,intensity
47453,47454,2,2221.261157,1111.134217,p0047454,E(304.2071453)SYVETELIFALAK(304.2071453);2;111...,sp|Q00610|CLH1_HUMAN,[L\tMod_Pos\t1\t304.2071453\tTMTpro 16plex Tan...,L\tSpectrum\thtmt_b309_f014.34678.2;htmt_b309_...,L\tJScore\t105.98;103.43;99.18;98.66;98.03;97....,105.98,L\tPSM#\t10,86.493,DongGeun/Boer/Danting TMT16 experiments htmt_b...,44,19,"[406.2621002, 434.2570149, 434.2934004, 451.31...","[1027090.0, 4846934.0, 151788.0, 7588905.0, 44..."


In [44]:
test.query('charge == {} & precursorMZ < {}'.format(z, check-8)).tail(1)

,scan,charge,[M+H]+,precursorMZ,L_ID,L_peptide,L_protein,mod_pos_line,spectrum_line,jscore_line,max_JScore,psm_no_line,RT,Lib_Description,Total_Batches,Peptide_observed,m/z,intensity
255925,255926,2,2189.259372,1095.133324,p0255926,T(304.2071453)K(304.2071453)PSEDSLSGQK(304.207...,sp|P51116|FXR2_HUMAN,[L\tMod_Pos\t1\t304.2071453\tTMTpro 16plex Tan...,L\tSpectrum\tSY_TMTpro_B6_d30x1ul_f02.9654.2,L\tJScore\t63.26,63.26,L\tPSM#\t1,11.199,Shu TMT16 MDD BM-10 htmt126,40,2,"[388.2515356, 406.2621002, 451.3199495, 562.35...","[3114.0, 26632.0, 86699.0, 2349.0, 14089.0, 31..."


In [28]:
test.query('precursorMZ > '+str(check_shift)+' & charge == '+z).head(1)

,scan,charge,[M+H]+,precursorMZ,L_ID,L_peptide,L_protein,mod_pos_line,spectrum_line,jscore_line,max_JScore,psm_no_line,RT,Lib_Description,Total_Batches,Peptide_observed,m/z,intensity
47453,47454,2,2221.261157,1111.134217,p0047454,E(304.2071453)SYVETELIFALAK(304.2071453);2;111...,sp|Q00610|CLH1_HUMAN,[L\tMod_Pos\t1\t304.2071453\tTMTpro 16plex Tan...,L\tSpectrum\thtmt_b309_f014.34678.2;htmt_b309_...,L\tJScore\t105.98;103.43;99.18;98.66;98.03;97....,105.98,L\tPSM#\t10,86.493,DongGeun/Boer/Danting TMT16 experiments htmt_b...,44,19,"[406.2621002, 434.2570149, 434.2934004, 451.31...","[1027090.0, 4846934.0, 151788.0, 7588905.0, 44..."


In [ ]:
df.query(' A > 3' ).head(1)

In [8]:
test_prec_mz = [1103.1334053, 735.758029, 1246.1466533]
z = [2,3,2]

In [57]:
#mz = list(decoy_spectrumDF["m/z"])[0]
#intensity = list(decoy_spectrumDF["intensity"])[0]
#                charge = int(row[mz_cols.index("charge")]
def getDecoySpectrum_SpectraST2(sortedDF, precMZ, decoy_search, charge, scan):
    
    if scan%2 == 1:
        decoy_search == -1*decoy_search
        check_shift = precMZ+decoy_search
        check = sortedDF.query('charge == {} & precursorMZ < {}'.format(charge, check_shift)).tail(1)
        if check.shape[0] == 0:
            decoy_search == -1*decoy_search
            check_shift = precMZ+decoy_search
            check = sortedDF.query('charge == {} & precursorMZ > {}'.format(charge, check_shift)).head(1)
    else:
        check_shift = precMZ+decoy_search
        check = sortedDF.query('charge == {} & precursorMZ > {}'.format(charge, check_shift)).head(1)
        if check.shape[0] == 0:
            decoy_search == -1*decoy_search
            check_shift = precMZ+decoy_search
            check = sortedDF.query('charge == {} & precursorMZ < {}'.format(charge, check_shift)).tail(1)

    mz_cols = list(check.columns)
    arr = check.to_numpy()
    row = arr[0]
    mz = list(row[mz_cols.index("m/z")])
    intensity = list(row[mz_cols.index("intensity")])
    scan_pair = int(row[mz_cols.index("scan")])
    precMZ_pair = float(row[mz_cols.index("precursorMZ")])
    L_ID_pair = row[mz_cols.index("L_ID")]
    L_peptide_pair = row[mz_cols.index("L_peptide")]
    L_protein_pair = row[mz_cols.index("L_protein")]
    RT_pair = float(row[mz_cols.index("RT")])
    
    return mz,intensity,scan_pair,precMZ_pair,L_ID_pair,L_peptide_pair,L_protein_pair,RT_pair

In [46]:
#mz = list(decoy_spectrumDF["m/z"])[0]
#intensity = list(decoy_spectrumDF["intensity"])[0]
#                charge = int(row[mz_cols.index("charge")]
def getDecoySpectrum_SpectraST(targetDF, precMZ, decoy_search, charge):
    
    #This function uses precMZ to scan over all library outside the +/-decoy search range but closest precMZ value
    #the precursor MZ is not between the range of decoy_search; Δ m/z > decoy_search. We want to have same charge. Now this will give us 100s of candidates
    
    reqd_cols = ["precursorMZ","charge","m/z","intensity","scan","L_ID","L_peptide","L_protein","RT"]
    #select by charge
    select_mz_DF_1 = targetDF.loc[targetDF["charge"] == charge]
    select_mz_DF_1["abs_delMZ"] = abs(np.array(select_mz_DF_1.precursorMZ)-precMZ)
    #apply cutoff |8Da|
    #Adding 200 Dalton as cutoff should help us to find at least 1 decoy candidate
    select_mz_DF=select_mz_DF_1.loc[select_mz_DF_1["abs_delMZ"].between((decoy_search),(decoy_search+10))]

    #make sure we have at least one decoy found
    while select_mz_DF.shape[0] < 1:
        decoy_search +=10
        select_mz_DF=select_mz_DF_1.loc[select_mz_DF_1["abs_delMZ"].between((decoy_search),(decoy_search+10))]
    
    
    #sort by ascending order of absolute difference 
    select_mz_DF2 = select_mz_DF.copy().sort_values(by=["abs_delMZ"], ascending=True)
    #select the first row
    check = select_mz_DF2.iloc[0:1]
    mz_cols = list(check.columns)
    arr = check.to_numpy()
    row = arr[0]
    mz = list(row[mz_cols.index("m/z")])
    intensity = list(row[mz_cols.index("intensity")])
    scan_pair = int(row[mz_cols.index("scan")])
    precMZ_pair = float(row[mz_cols.index("precursorMZ")])
    L_ID_pair = row[mz_cols.index("L_ID")]
    L_peptide_pair = row[mz_cols.index("L_peptide")]
    L_protein_pair = row[mz_cols.index("L_protein")]
    RT_pair = float(row[mz_cols.index("RT")])
    
    return mz,intensity,scan_pair,precMZ_pair,L_ID_pair,L_peptide_pair,L_protein_pair,RT_pair

In [47]:
test.columns

Index(['scan', 'charge', '[M+H]+', 'precursorMZ', 'L_ID', 'L_peptide',
       'L_protein', 'mod_pos_line', 'spectrum_line', 'jscore_line',
       'max_JScore', 'psm_no_line', 'RT', 'Lib_Description', 'Total_Batches',
       'Peptide_observed', 'm/z', 'intensity'],
      dtype='object')

In [48]:
getDecoySpectrum2(targetDF, 1246.1466533, 8, 2)

NameError: name 'getDecoySpectrum2' is not defined

In [58]:
def decoySpecLibrary_Prec_Swap_Pair_Dict(df, df_sort, specLibFolder, d, libtypename):
    mz_cols = list(df.columns)
    np_arr = df.to_numpy()

    scan_cnt = df.shape[0]
   
    proton = 1.00727646677
    write_log ("  Generating .ms2 files\n")
    now = datetime.now()
    #print (prev_mod)
    write_log("  now =", now)
    
    write_log ("  Decoy spectral library dictionary is being created")
    
    exclusion_list = []
    decoy_dictionary = {"scan":[], "Z":[],"L_ID":[],"L_peptide":[],"L_protein":[],
                       "massNeutral":[],"precMZ":[],"RT":[],"mzArray":[],"intArray":[]}
    
        
    for row in np_arr:
        scan = int(row[mz_cols.index("scan")])
        decoy_scan = scan+scan_cnt+len(exclusion_list)
        if scan not in exclusion_list:
            charge = int(row[mz_cols.index("charge")])
            precMZ = float(row[mz_cols.index("precursorMZ")])
            L_ID = "Decoy_"+row[mz_cols.index("L_ID")]
            L_peptide = "Decoy_"+row[mz_cols.index("L_peptide")]
            L_protein = "Decoy_"+row[mz_cols.index("L_protein")] 


            mz,intensity,scan_pair,precMZ_pair,L_ID_pair,L_peptide_pair,L_protein_pair,RT_pair = getDecoySpectrum_SpectraST2(df_sort, precMZ, d, charge, scan)

            RT = float(row[mz_cols.index("RT")])
            massNeutral = (precMZ*charge) - ((charge-1)*proton)

            decoy_dictionary["scan"].append(scan)
            decoy_dictionary["Z"].append(charge)
            decoy_dictionary["L_ID"].append(L_ID)
            decoy_dictionary["L_peptide"].append(L_peptide)
            decoy_dictionary["L_protein"].append(L_protein)
            decoy_dictionary["massNeutral"].append(massNeutral)
            decoy_dictionary["precMZ"].append(precMZ)
            decoy_dictionary["RT"].append(RT)
            decoy_dictionary["mzArray"].append(mz)
            decoy_dictionary["intArray"].append(intensity)


            scan_pair = str(decoy_scan+1)

            mz_pair = list(row[mz_cols.index("m/z")])
            intensity_pair = list(row[mz_cols.index("intensity")])

            massNeutral_pair = (precMZ_pair*charge) - ((charge-1)*proton)

            decoy_dictionary["scan"].append(scan_pair)
            decoy_dictionary["Z"].append(charge)
            decoy_dictionary["L_ID"].append(L_ID_pair)
            decoy_dictionary["L_peptide"].append(L_peptide_pair)
            decoy_dictionary["L_protein"].append(L_protein_pair)
            decoy_dictionary["massNeutral"].append(massNeutral_pair)
            decoy_dictionary["precMZ"].append(precMZ_pair)
            decoy_dictionary["RT"].append(RT_pair)
            decoy_dictionary["mzArray"].append(mz_pair)
            decoy_dictionary["intArray"].append(intensity_pair)

            exclusion_list.append(scan)
            if np.mod((len(exclusion_list)),1000) == 0:
                print ("Total scans completed {}".format(len(exclusion_list)))


        else:
            pass

    write_log ("  Done ...\n")
    return decoy_dictionary

In [ ]:
def decoySpecLibrary_Prec_Swap_Pair(df, df_sort, specLibFolder, d, libtypename):
    mz_cols = list(df.columns)
    np_arr = df.to_numpy()

    scan_cnt = df.shape[0]
   
    proton = 1.00727646677
    write_log ("  Generating .ms2 files\n")
    now = datetime.now()
    #print (prev_mod)
    write_log("  now =", now)
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%m/%d/%Y %H:%M %p")
    year = now.strftime("%Y")
    date_time = dt_string.split() #date is 0 index, time is 1 index, AM/PM is 2 index
    
    header_ms2 = "H\tCreationDate\t"+dt_string+"\nH\tExtractor\tMakeMS2\nH\tExtractorVersion\t1.0\nH\tComments\tMakeMS2 written by Suresh Poudel, "+year+"\nH\tExtractorOptions\tMS2/MS1\n"
    new_ms2_file = specLibFolder+"/intermediate/jumplib_human_{}_decoy.splib".format(libtypename)
    # new_ms2_file = specLibFolder+"/Spectral_Library_Decoy.spLib" #modified the extension name to ms2pep
    write_log ("  Decoy spectral library is being created")
    
    exclusion_list = []
    
    
    with open(new_ms2_file,"w") as new_ms2:
        new_ms2.write(header_ms2)
        
        for row in np_arr:
            scan = int(row[mz_cols.index("scan")])
            decoy_scan = scan+scan_cnt+len(exclusion_list)
            if scan not in exclusion_list:
                charge = int(row[mz_cols.index("charge")])
                precMZ = float(row[mz_cols.index("precursorMZ")])
                L_ID = "Decoy_"+row[mz_cols.index("L_ID")]
                L_peptide = "Decoy_"+row[mz_cols.index("L_peptide")]
                L_protein = "Decoy_"+row[mz_cols.index("L_protein")] 


                mz,intensity,scan_pair,precMZ_pair,L_ID_pair,L_peptide_pair,L_protein_pair,RT_pair = getDecoySpectrum_SpectraST2(df_sort, precMZ, d, charge, scan)
                
                RT = float(row[mz_cols.index("RT")])
                massNeutral = (precMZ*charge) - ((charge-1)*proton)
                

               
                mz_pair = list(row[mz_cols.index("m/z")])
                intensity_pair = list(row[mz_cols.index("intensity")])
                
                massNeutral_pair = (precMZ_pair*charge) - ((charge-1)*proton)

                
                new_ms2.write("S\t"+str(decoy_scan)+"\t"+str(decoy_scan)+"\t"+str(precMZ)+"\n")
                new_ms2.write("Z\t"+str(charge)+"\t"+str(massNeutral)+"\n")
                new_ms2.write("L\tID_with_Modification\t"+L_ID+"\t"+L_peptide+"\n")
                new_ms2.write("L\tProtein\tRepresentative\t"+L_protein+"\n")

                new_ms2.write("L\tMH+_Decoy\t"+str(massNeutral)+"\n")
                new_ms2.write("L\tPrecursor_Decoy\tz="+str(charge)+"\t"+str(precMZ)+"\n")
                new_ms2.write("L\tRT_Decoy\t"+str(RT)+"\n")

                for index, val in enumerate(mz):
                    new_ms2.write(str(val)+"\t"+str(intensity[index])+"\n")
                
                scan_pair = str(decoy_scan+1)
                new_ms2.write("S\t"+scan_pair+"\t"+scan_pair+"\t"+str(precMZ_pair)+"\n")
                new_ms2.write("Z\t"+str(charge)+"\t"+str(massNeutral_pair)+"\n")
                new_ms2.write("L\tID_with_Modification\t"+L_ID_pair+"\t"+L_peptide_pair+"\n")
                new_ms2.write("L\tProtein\tRepresentative\t"+L_protein_pair+"\n")

                new_ms2.write("L\tMH+_Decoy\t"+str(massNeutral_pair)+"\n")
                new_ms2.write("L\tPrecursor_Decoy\tz="+str(charge)+"\t"+str(precMZ_pair)+"\n")
                new_ms2.write("L\tRT_Decoy\t"+str(RT_pair)+"\n")

                for index_p, val_p in enumerate(mz_pair):
                    new_ms2.write(str(val_p)+"\t"+str(intensity_pair[index_p])+"\n")
                
                
                exclusion_list.append(scan)
                if np.mod((len(exclusion_list)),10000) == 0:
                    print ("Total scans completed {}".format(len(exclusion_list)))
                
                
            else:
                pass
            
    write_log ("  Done ...\n")

In [64]:
def decoySpecLibrary_Prec_Swap(df, specLibFolder, d, libtypename):
    mz_cols = list(df.columns)
    np_arr = df.to_numpy()

    scan_cnt = df.shape[0]
   
    proton = 1.00727646677
    write_log ("  Generating .ms2 files\n")
    now = datetime.now()
    #print (prev_mod)
    write_log("  now =", now)
    # dd/mm/YY H:M:S
    dt_string = now.strftime("%m/%d/%Y %H:%M %p")
    year = now.strftime("%Y")
    date_time = dt_string.split() #date is 0 index, time is 1 index, AM/PM is 2 index
    
    header_ms2 = "H\tCreationDate\t"+dt_string+"\nH\tExtractor\tMakeMS2\nH\tExtractorVersion\t1.0\nH\tComments\tMakeMS2 written by Suresh Poudel, "+year+"\nH\tExtractorOptions\tMS2/MS1\n"
    new_ms2_file = specLibFolder+"/intermediate/jumplib_human_{}_decoy.splib".format(libtypename)
    # new_ms2_file = specLibFolder+"/Spectral_Library_Decoy.spLib" #modified the extension name to ms2pep
    write_log ("  Decoy spectral library is being created")
    
    with open(new_ms2_file,"w") as new_ms2:
        new_ms2.write(header_ms2)
        
        for row in np_arr:
            scan = str(int(row[mz_cols.index("scan")])+scan_cnt)
            charge = int(row[mz_cols.index("charge")])
            precMZ = float(row[mz_cols.index("precursorMZ")])
            L_ID = "Decoy_"+row[mz_cols.index("L_ID")]
            L_peptide = "Decoy_"+row[mz_cols.index("L_peptide")]
            L_protein = "Decoy_"+row[mz_cols.index("L_protein")] 
            
            precursorMZ = precMZ
            
            decoy_spectrumDF = getDecoySpectrum_SpectraST(df, precMZ, d, charge)
        
            
            mz = list(decoy_spectrumDF["m/z"])[0]
            intensity = list(decoy_spectrumDF["intensity"])[0]
            

            RT = float(row[mz_cols.index("RT")])

            massNeutral = (precursorMZ*charge) - ((charge-1)*proton)
            

            new_ms2.write("S\t"+scan+"\t"+scan+"\t"+str(precursorMZ)+"\n")
            new_ms2.write("Z\t"+str(charge)+"\t"+str(massNeutral)+"\n")
            new_ms2.write("L\tID_with_Modification\t"+L_ID+"\t"+L_peptide+"\n")
            new_ms2.write("L\tProtein\tRepresentative\t"+L_protein+"\n")
            
            new_ms2.write("L\tMH+_Decoy\t"+str(massNeutral)+"\n")
            new_ms2.write("L\tPrecursor_Decoy\tz="+str(charge)+"\t"+str(precursorMZ)+"\n")
            new_ms2.write("L\tRT_Decoy\t"+str(RT)+"\n")

            for index, val in enumerate(mz):
                new_ms2.write(str(val)+"\t"+str(intensity[index])+"\n")
            
    write_log ("  Done ...\n")

In [48]:
specLibFolder = "../../Decoy_Precursor_Swap"
libtypename = "default"

In [ ]:
d=8
decoySpecLibrary_Prec_Swap_Pair_Dict(targetDF, test, specLibFolder, d, libtypename)

  Generating .ms2 files

  now = 2021-12-01 11:51:19.648095
  Decoy spectral library dictionary is being created
Total scans completed 1000
Total scans completed 2000
Total scans completed 3000
Total scans completed 4000
Total scans completed 5000
Total scans completed 6000
Total scans completed 7000
Total scans completed 8000
Total scans completed 9000
Total scans completed 10000
Total scans completed 11000
Total scans completed 12000
Total scans completed 13000
Total scans completed 14000
Total scans completed 15000
Total scans completed 16000
Total scans completed 17000
Total scans completed 18000
Total scans completed 19000
Total scans completed 20000
Total scans completed 21000
Total scans completed 22000
Total scans completed 23000
Total scans completed 24000
Total scans completed 25000
Total scans completed 26000
Total scans completed 27000
Total scans completed 28000
Total scans completed 29000
Total scans completed 30000
Total scans completed 31000
Total scans completed 32000


Total scans completed 284000
Total scans completed 285000
Total scans completed 286000
Total scans completed 287000
Total scans completed 288000
Total scans completed 289000
Total scans completed 290000
Total scans completed 291000
Total scans completed 292000
Total scans completed 293000
Total scans completed 294000
Total scans completed 295000


In [68]:
np.mod(10,3)

1